# Word Embeddings First Steps: Data Preparation

In this series of ungraded notebooks, you'll try out all the individual techniques that you learned about in the lectures. Practicing on small examples will prepare you for the graded assignment, where you will combine the techniques in more advanced ways to create word embeddings from a real-life corpus. 

This notebook focuses on data preparation, which is the first step of any machine learning algorithm. It is a very important step because models are only as good as the data they are trained on and the models used require the data to have a particular structure to process it properly.

To get started, import and initialize all the libraries you will need.

In [2]:
%%capture
!pip install -Uqq emoji

In [3]:
import re
import nltk
import emoji
import numpy as np
from nltk.tokenize import word_tokenize

nltk.download('punkt')

print("nltk  ", nltk.__version__)
print("emoji ", emoji.__version__)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
nltk   3.2.5
emoji  0.6.0


In [4]:
def get_dict(data):
    """
    Input:
        K: the number of negative samples
        data: the data you want to pull from
        indices: a list of word indices
    Output:
        word_dict: a dictionary with the weighted probabilities of each word
        word2Ind: returns dictionary mapping the word to its index
        Ind2Word: returns dictionary mapping the index to its word
    """
    words = sorted(list(set(data)))
    n = len(words)
    idx = 0
    # return these correctly
    word2Ind = {}
    Ind2word = {}
    for k in words:
        word2Ind[k] = idx
        Ind2word[idx] = k
        idx += 1
    return word2Ind, Ind2word

## Data preparation

In the data preparation phase, starting with a corpus of text, you will:

- Clean and tokenize the corpus.

- Extract the pairs of context words and center word that will make up the training data set for the CBOW model. The context words are the features that will be fed into the model, and the center words are the target values that the model will learn to predict.

- Create simple vector representations of the context words (features) and center words (targets) that can be used by the neural network of the CBOW model.

### Cleaning and tokenization

To demonstrate the cleaning and tokenization process, consider a corpus that contains emojis and various punctuation signs.

In [5]:
# Define a corpus
corpus = 'Who ❤️ "word embeddings" in 2020? I do!!! I love pre-processing 2:)!?'

First, replace all interrupting punctuation signs — such as commas and exclamation marks — with periods.

In [6]:
print("Corpus:\n\t", corpus, "\n")

data = re.sub(r"[,!?;-]+", ".", corpus)

print("After cleaning punctuation:\n\t", data)

Corpus:
	 Who ❤️ "word embeddings" in 2020? I do!!! I love pre-processing 2:)!? 

After cleaning punctuation:
	 Who ❤️ "word embeddings" in 2020. I do. I love pre.processing 2:).


Next, use NLTK's tokenization engine to split the corpus into individual tokens.

In [7]:
print("Initial string:\n\t", data, "\n")
tokenized = nltk.word_tokenize(data)
print("After tokenization\n\t", tokenized)

Initial string:
	 Who ❤️ "word embeddings" in 2020. I do. I love pre.processing 2:). 

After tokenization
	 ['Who', '❤️', '``', 'word', 'embeddings', "''", 'in', '2020', '.', 'I', 'do', '.', 'I', 'love', 'pre.processing', '2', ':', ')', '.']


Finally, as you saw in the lecture, get rid of numbers and punctuation other than periods, and convert all the remaining tokens to lowercase.

In [8]:
print("Initial list of tokens:\n\t", tokenized, "\n")

clean_tokenized = [
    ch.lower()
    for ch in tokenized
    if ch.isalpha() or ch == "." or emoji.get_emoji_regexp().search(ch)
]

print("After cleaning\n\t", clean_tokenized)

Initial list of tokens:
	 ['Who', '❤️', '``', 'word', 'embeddings', "''", 'in', '2020', '.', 'I', 'do', '.', 'I', 'love', 'pre.processing', '2', ':', ')', '.'] 

After cleaning
	 ['who', '❤️', 'word', 'embeddings', 'in', '.', 'i', 'do', '.', 'i', 'love', '.']


Note that the heart emoji is considered as a token just like any normal word.

Now let's streamline the cleaning and tokenization process by wrapping the previous steps in a function.

In [9]:
def tokenize(corpus):
    data = re.sub(r"[,!?;-]+", ".", corpus)
    data = nltk.word_tokenize(data)
    data = [
        ch.lower()
        for ch in data
        if ch.isalpha() or ch == "." or emoji.get_emoji_regexp().search(ch)
    ]
    return data

Apply this function to the corpus that you'll be working on in the rest of this notebook: "I am happy because I am learning"

In [10]:
corpus = "I am happy because I am learning"

print(f'Corpus:  {corpus}')
words = tokenize(corpus)
print(f'Words (tokens):  {words}')

Corpus:  I am happy because I am learning
Words (tokens):  ['i', 'am', 'happy', 'because', 'i', 'am', 'learning']


## Sliding window of words

Now that you have transformed the corpus into a list of clean tokens, you can slide a window of words across this list. For each window you can extract a center word and the context words.

The `get_windows` function in the next cell was introduced in the lecture.

In [11]:
def get_windows(words, C):
    i = C
    while i < len(words) - C:
        center_word = words[i]
        context_words = words[i - C : i] + words[i + 1 : i + 1 + C]
        yield context_words, center_word
        i += 1

The first argument of this function is a list of words (or tokens). The second argument, `C`, is the context half-size. Recall that for a given center word, the context words are made of `C` words to the left and `C` words to the right of the center word.

Here is how you can use this function to extract context words and center words from a list of tokens. These context and center words will make up the training set that you will use to train the CBOW model.

In [12]:
for x, y in get_windows(
    ["i", "am", "happy", "because", "i", "am", "learning"], 2
):
    print(f"{x}\t{y}")

['i', 'am', 'because', 'i']	happy
['am', 'happy', 'i', 'am']	because
['happy', 'because', 'am', 'learning']	i


The first example of the training set is made of:

- the context words "i", "am", "because", "i",

- and the center word to be predicted: "happy".

**Now try it out yourself. In the next cell, you can change both the sentence and the context half-size.**

In [13]:
for x, y in get_windows(
    tokenize("Now it's your turn: try with your own sentence!"), 1
):
    print(f"{x}\t{y}")

['now', 'your']	it
['it', 'turn']	your
['your', 'try']	turn
['turn', 'with']	try
['try', 'your']	with
['with', 'own']	your
['your', 'sentence']	own
['own', '.']	sentence


## Transforming words into vectors for the training set

To finish preparing the training set, you need to transform the context words and center words into vectors.

### Mapping words to indices and indices to words

The center words will be represented as one-hot vectors, and the vectors that represent context words are also based on one-hot vectors.

To create one-hot word vectors, you can start by mapping each unique word to a unique integer (or index). We have provided a helper function, `get_dict`, that creates a Python dictionary that maps words to integers and back.

In [14]:
word2ind, ind2word = get_dict(words)
word2ind, ind2word

({'am': 0, 'because': 1, 'happy': 2, 'i': 3, 'learning': 4},
 {0: 'am', 1: 'because', 2: 'happy', 3: 'i', 4: 'learning'})

In [15]:
V = len(word2ind)

print("Size of vocabulary", V)

Size of vocabulary 5


### Getting one-hot word vectors

Recall from the lecture that you can easily convert an integer, $n$, into a one-hot vector.

Consider the word "happy". First, retrieve its numeric index.

In [16]:
n = word2ind["happy"]
n

2

In [19]:
center_word_vector = np.zeros(V)
center_word_vector[n] = 1
center_word_vector

array([0., 0., 1., 0., 0.])

In [20]:
def word_to_one_hot_vector(word, word2ind, V):
    one_hot_vector = np.zeros(V)
    one_hot_vector[word2ind[word]] = 1
    return one_hot_vector

In [21]:
word_to_one_hot_vector("happy", word2ind, V)

array([0., 0., 1., 0., 0.])

### Getting context word vectors

To create the vectors that represent context words, you will calculate the average of the one-hot vectors representing the individual words.

Let's start with a list of context words.

In [22]:
context_words = ['i', 'am', 'because', 'i']

In [23]:
context_words_vectors = [word_to_one_hot_vector(w, word2ind, V) for w in context_words]
context_words_vectors

[array([0., 0., 0., 1., 0.]),
 array([1., 0., 0., 0., 0.]),
 array([0., 1., 0., 0., 0.]),
 array([0., 0., 0., 1., 0.])]

And you can now simply get the average of these vectors using numpy's `mean` function, to get the vector representation of the context words.

In [24]:
np.mean(context_words_vectors, axis=0)

array([0.25, 0.25, 0.  , 0.5 , 0.  ])

And you can now simply get the average of these vectors using numpy's `mean` function, to get the vector representation of the context words.

In [26]:
np.mean(context_words_vectors, axis=1)

array([0.2, 0.2, 0.2, 0.2])

Note the `axis=0` parameter that tells `mean` to calculate the average of the rows (if you had wanted the average of the columns, you would have used `axis=1`).

**Now create the `context_words_to_vector` function that takes in a list of context words, a word-to-index dictionary, and a vocabulary size, and outputs the vector representation of the context words.**

In [ ]:
def 